In [10]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import sqlite3
import os
from tqdm import tqdm
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt

In [12]:
test_fname = "/home/boris/Data/CrossDocked/types/it2_tt_v1.1_0_test0.types"
test_df = pd.read_csv(test_fname, sep=' ', names=["label", "binding_affinity", "crystal_rmsd", "rec_file", "lig_file", "vina_score"])

In [13]:
test_df

,label,binding_affinity,crystal_rmsd,rec_file,lig_file,vina_score
0,0,-0.0,7.14898,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_0.gninatypes,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_2gj6_3ib_li...,#196.41899
1,0,-0.0,6.66502,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_0.gninatypes,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_2gj6_3ib_li...,#52.25949
2,0,-0.0,6.66188,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_0.gninatypes,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_2gj6_3ib_li...,#96.62679
3,0,-0.0,13.43570,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_0.gninatypes,1A02_HUMAN_25_199_pep_0/1ao7_A_rec_2gj6_3ib_li...,#200.97296
4,0,-0.0,11.59660,1A02_HUMAN_25_199_pep_0/4uq3_C_rec_0.gninatypes,1A02_HUMAN_25_199_pep_0/4uq3_C_rec_2gj6_3ib_li...,#133.94607
...,...,...,...,...,...,...
8015439,0,-0.0,5.88147,ZFPS_SOLHA_71_303_0/5hxn_A_rec_0.gninatypes,ZFPS_SOLHA_71_303_0/5hxn_A_rec_5hxt_dst_lig_tt...,#-5.92324
8015440,0,-0.0,6.77640,ZFPS_SOLHA_71_303_0/5hxn_A_rec_0.gninatypes,ZFPS_SOLHA_71_303_0/5hxn_A_rec_5hxt_dst_lig_tt...,#-5.47303
8015441,0,-0.0,4.70560,ZFPS_SOLHA_71_303_0/5hxn_A_rec_0.gninatypes,ZFPS_SOLHA_71_303_0/5hxn_A_rec_5hxt_dst_lig_tt...,#-5.36538
8015442,0,-0.0,6.16644,ZFPS_SOLHA_71_303_0/5hxn_A_rec_0.gninatypes,ZFPS_SOLHA_71_303_0/5hxn_A_rec_5hxt_dst_lig_tt...,#-5.02480


In [14]:
fname = "/home/boris/Data/SIFTS/pdb_chain_uniprot.csv"
sifts_df = pd.read_csv(fname, comment='#')
sifts_df

con = sqlite3.connect("/home/boris/Data/ChemBL/chembl_30/chembl_30_sqlite/chembl_30.db")
cursor = con.cursor()
cursor.execute("create table if not exists sifts (pdb text, chain text sp_primary text, res_beg integer, res_end integer, pdb_beg integer, pdb_end integer, sp_beg integer, sp_end integer)")
cursor.fetchall()

sifts_df.to_sql('sifts', con, if_exists='replace', index=False)
pd.read_sql_query("select * from sifts", con)

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,101m,A,P02185,1,154,0,153,1,154
1,102l,A,P00720,1,40,1,40,1,40
2,102l,A,P00720,42,165,41,None,41,164
3,102m,A,P02185,1,154,0,153,1,154
4,103l,A,P00720,1,40,1,None,1,40
...,...,...,...,...,...,...,...,...,...
635822,9xia,A,P24300,1,388,1,None,1,388
635823,9xim,A,P12851,1,393,None,394,2,394
635824,9xim,B,P12851,1,393,None,394,2,394
635825,9xim,C,P12851,1,393,None,394,2,394


In [15]:
test_pdbs = test_df['rec_file'].apply(lambda f: f.split('/')[-1].split('_')[0])
test_pdbs
test_pdbs_str = ", ".join(map(lambda s: f"'{s}'", test_pdbs))

test_uniprots = pd.read_sql_query(f"select SP_PRIMARY from sifts where PDB in ({test_pdbs_str})", con)
test_uniprots

,SP_PRIMARY
0,P02185
1,P12931
2,P12931
3,P12931
4,P12931
...,...
21784,P61823
21785,P12851
21786,P12851
21787,P12851


In [20]:
activities = pd.read_csv("/home/boris/Data/BigBind/activities.csv")

In [16]:
struct_fname = "/home/boris/Data/BigBind/structures.csv"
struct_df = pd.read_csv(struct_fname)
struct_df

,uniprot,lig_file,rec_file,num_rec_residues,lig_rdkit_loadable,lig_rdkit_embeddable
0,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1b0g_A_rec_pocket.pdb,14,True,True
1,P04439,P04439/2gj6_3ib_lig.sdf,P04439/2x4s_A_rec_pocket.pdb,10,True,True
2,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1t1w_A_rec_pocket.pdb,13,True,True
3,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1s9y_A_rec_pocket.pdb,14,True,True
4,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1jf1_A_rec_pocket.pdb,16,True,True
...,...,...,...,...,...,...
701159,P77173,P77173/1s1s_wac_lig.sdf,P77173/1f47_B_rec_pocket.pdb,50,True,True
701160,P77173,P77173/1s1j_iqz_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True
701161,P77173,P77173/1y2g_cl3_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True
701162,P77173,P77173/1y2f_wai_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True


In [17]:
test_uniprots = set(test_uniprots["SP_PRIMARY"])

In [21]:
activities["is_test"] = activities["protein_accession"].apply(lambda u: u in test_uniprots)

In [18]:
struct_df["is_test"] = struct_df["uniprot"].apply(lambda u: u in test_uniprots)
struct_df

,uniprot,lig_file,rec_file,num_rec_residues,lig_rdkit_loadable,lig_rdkit_embeddable,is_test
0,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1b0g_A_rec_pocket.pdb,14,True,True,True
1,P04439,P04439/2gj6_3ib_lig.sdf,P04439/2x4s_A_rec_pocket.pdb,10,True,True,True
2,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1t1w_A_rec_pocket.pdb,13,True,True,True
3,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1s9y_A_rec_pocket.pdb,14,True,True,True
4,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1jf1_A_rec_pocket.pdb,16,True,True,True
...,...,...,...,...,...,...,...
701159,P77173,P77173/1s1s_wac_lig.sdf,P77173/1f47_B_rec_pocket.pdb,50,True,True,False
701160,P77173,P77173/1s1j_iqz_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True,False
701161,P77173,P77173/1y2g_cl3_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True,False
701162,P77173,P77173/1y2f_wai_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True,False


In [19]:
struct_df.to_csv(struct_fname, index=False)

In [22]:
filename = "/home/boris/Data/BigBind/activities.csv"
activities.to_csv(filename, header=True, index=False)